In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference_replisome.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
eval_params = param.Evaluation.to_dict()
train_size = simulation_params['train_size']

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.init_recorder()
model.init_eval_data()

Eval image generation: 100%|████████████████████████████████████████| 30/30 [00:13<00:00,  2.26it/s]


eval images shape: (30, 1041, 1302) contain 23327 molecules,


In [8]:
model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [9]:
simulation_params

{'train_size': 128,
 'train_type': 'cells',
 'psf_intensity_normalization': True,
 'density': 0.00625,
 'non_cell_density': 10,
 'margin_empty': 0.05,
 'z_prior': [-1, 0.5],
 'survival_prob': None,
 'min_photon': 0.25,
 'camera': 'sCMOS',
 'camera_chip_size': [1041, 1302],
 'perlin_noise': False,
 'perlin_noise_factor': 0.2,
 'perlin_noise_res': 64,
 'baseline': 103.0,
 'bg_values': 112.0,
 'e_per_adu': 0.38,
 'qe': 0.95,
 'spurious_c': 0.0,
 'sig_read': '/mnt/sda1/SMLAT/data/real_data/dark_images/RN_e_venus_upgraded.tiff',
 'robust_training': False,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/replisome_dots/pool_3051_repli_pos01/phase_venus_mask/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/replisome_dots/pool_3051_repli_pos01/edt_noise.pkl',
 'min_edt': 0,
 'max_edt': 6,
 'augment': True}

In [10]:
start_time = time.time()
model.fit(batch_size=64, max_iters=20000, print_output=True, print_freq=100)
duration = time.time() - start_time

Started training .... 
 || Cost: 5628.674 || Time Upd.: 924.1 ms  || BatchNr.: 100
 || Cost: 517.486 || Time Upd.: 894.3 ms  || BatchNr.: 200
 || Cost: 241.222 || Time Upd.: 903.1 ms  || BatchNr.: 300
 || Cost: 192.020 || Time Upd.: 901.9 ms  || BatchNr.: 400
 || Cost: 130.195 || Time Upd.: 906.3 ms  || BatchNr.: 500
 || Cost: 115.704 || Time Upd.: 904.1 ms  || BatchNr.: 600
 || Cost: 105.773 || Time Upd.: 904.4 ms  || BatchNr.: 700
 || Cost: 98.431 || Time Upd.: 905.5 ms  || BatchNr.: 800
 || Cost: 88.736 || Time Upd.: 905.6 ms  || BatchNr.: 900
 || Cost: 92.868 || Time Upd.: 905.4 ms  || BatchNr.: 1000
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 11854
JoR: 1.044 || Eff_3d: 29.374 || Jaccard: 0.508 || Factor: 728.832 || RMSE_lat: 48.642 || RMSE_ax: 105.218 || RMSE_x: 

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 18183
JoR: 1.596 || Eff_3d: 44.259 || Jaccard: 0.774 || Factor: 833.959 || RMSE_lat: 48.518 || RMSE_ax: 106.769 || RMSE_x: 34.996 || RMSE_y: 33.605 || Cost: 57.388 || Recall: 0.777 || Precision: 0.995 || BatchNr.: 2200 || Time Upd.: 905.1 ms 
 || Count Cost: 32.2700 || Localization Cost: 33.5237 || Bg Cost: 0.0996 || P_locs Cost: 37.7738saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 44.259 || 48.518 || 106.769 || 2200


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 18087
JoR: 1.486 || Eff_3d: 45

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 18235
JoR: 1.665 || Eff_3d: 50.548 || Jaccard: 0.778 || Factor: 735.558 || RMSE_lat: 46.731 || RMSE_ax: 83.288 || RMSE_x: 33.516 || RMSE_y: 32.565 || Cost: 49.703 || Recall: 0.780 || Precision: 0.997 || BatchNr.: 3400 || Time Upd.: 886.1 ms 
 || Count Cost: 30.9747 || Localization Cost: 28.6292 || Bg Cost: 0.0752 || P_locs Cost: 35.4492saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 50.548 || 46.731 || 83.288 || 3400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 19308
JoR: 1.669 || Eff_3d: 50.0

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 19419
JoR: 1.723 || Eff_3d: 51.703 || Jaccard: 0.827 || Factor: 826.186 || RMSE_lat: 47.977 || RMSE_ax: 84.311 || RMSE_x: 34.383 || RMSE_y: 33.461 || Cost: 48.646 || Recall: 0.830 || Precision: 0.995 || BatchNr.: 4600 || Time Upd.: 953.1 ms 
 || Count Cost: 32.6913 || Localization Cost: 27.2555 || Bg Cost: 0.0755 || P_locs Cost: 35.2578saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 51.703 || 47.977 || 84.311 || 4600


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 19660
JoR: 1.739 || Eff_3d: 51.1

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 19010
JoR: 1.752 || Eff_3d: 52.402 || Jaccard: 0.810 || Factor: 740.099 || RMSE_lat: 46.228 || RMSE_ax: 82.082 || RMSE_x: 33.492 || RMSE_y: 31.864 || Cost: 47.018 || Recall: 0.813 || Precision: 0.996 || BatchNr.: 5800 || Time Upd.: 974.5 ms 
 || Count Cost: 31.7599 || Localization Cost: 26.1647 || Bg Cost: 0.0726 || P_locs Cost: 34.7827saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 52.402 || 46.228 || 82.082 || 5800


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 19082
JoR: 1.754 || Eff_3d: 51.6

...

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 19569
JoR: 1.863 || Eff_3d: 55.273 || Jaccard: 0.836 || Factor: 748.625 || RMSE_lat: 44.860 || RMSE_ax: 76.636 || RMSE_x: 32.299 || RMSE_y: 31.132 || Cost: 42.406 || Recall: 0.838 || Precision: 0.997 || BatchNr.: 14200 || Time Upd.: 886.8 ms 
 || Count Cost: 30.5080 || Localization Cost: 22.4743 || Bg Cost: 0.0689 || P_locs Cost: 33.2837saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 55.273 || 44.860 || 76.636 || 14200


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 19950
JoR: 1.845 || Eff_3d: 54

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 19912
JoR: 1.830 || Eff_3d: 54.269 || Jaccard: 0.848 || Factor: 807.079 || RMSE_lat: 46.311 || RMSE_ax: 79.808 || RMSE_x: 33.164 || RMSE_y: 32.325 || Cost: 38.388 || Recall: 0.851 || Precision: 0.995 || BatchNr.: 15400 || Time Upd.: 930.8 ms 
 || Count Cost: 30.2387 || Localization Cost: 20.1925 || Bg Cost: 0.0496 || P_locs Cost: 32.7303saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 54.269 || 46.311 || 79.808 || 15400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 19773
JoR: 1.879 || Eff_3d: 55

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 19972
JoR: 1.886 || Eff_3d: 55.422 || Jaccard: 0.852 || Factor: 809.324 || RMSE_lat: 45.154 || RMSE_ax: 77.772 || RMSE_x: 32.655 || RMSE_y: 31.186 || Cost: 38.695 || Recall: 0.854 || Precision: 0.996 || BatchNr.: 16600 || Time Upd.: 886.1 ms 
 || Count Cost: 30.1675 || Localization Cost: 20.3099 || Bg Cost: 0.0517 || P_locs Cost: 32.8736saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 55.422 || 45.154 || 77.772 || 16600


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 19885
JoR: 1.862 || Eff_3d: 55

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 19922
JoR: 1.843 || Eff_3d: 54.772 || Jaccard: 0.851 || Factor: 775.795 || RMSE_lat: 46.159 || RMSE_ax: 78.374 || RMSE_x: 33.182 || RMSE_y: 32.089 || Cost: 38.604 || Recall: 0.853 || Precision: 0.997 || BatchNr.: 17800 || Time Upd.: 882.2 ms 
 || Count Cost: 29.8998 || Localization Cost: 20.4403 || Bg Cost: 0.0505 || P_locs Cost: 32.5645saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 54.772 || 46.159 || 78.374 || 17800


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 19472
JoR: 1.848 || Eff_3d: 54

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 19513
JoR: 1.894 || Eff_3d: 55.781 || Jaccard: 0.835 || Factor: 758.690 || RMSE_lat: 44.055 || RMSE_ax: 75.859 || RMSE_x: 32.001 || RMSE_y: 30.279 || Cost: 38.123 || Recall: 0.836 || Precision: 0.998 || BatchNr.: 19000 || Time Upd.: 929.7 ms 
 || Count Cost: 30.0869 || Localization Cost: 19.9584 || Bg Cost: 0.0496 || P_locs Cost: 32.7033saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 55.781 || 44.055 || 75.859 || 19000


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 23327FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 23293 ,preds: 19836
JoR: 1.896 || Eff_3d: 55

In [12]:
duration

25933.91123843193